# Chapter 17: Practical statistical modeling

In [ ]:
library(car)
library(tidyverse)
library(ggplot2)
library(emmeans)
library(cowplot)
library(knitr)
# library(ggfortify)
library(gplots)
# library(matlab)
library(dendextend)
library(psych)
library(MuMIn)
library(lme4)
library(lmerTest)
library(broom)
library(DHARMa)
library(ggbeeswarm)
library(viridis)
library(influence.ME)
library(broom.mixed)
library(ggExtra)

theme_set(theme_minimal(base_size = 14))

set.seed(123456) # set random seed to exactly replicate results

## Figure 17.1 Outliers and influential observations

In [ ]:
set.seed(1234)
npts = 24
outlier_df = data.frame(Y = rnorm(npts, 0, 1)) %>%
  mutate(X = rnorm(npts, 0, 1) + Y*0.4)

lm.result = lm(Y ~ X, data=outlier_df)
model.data = augment(lm.result)
p2 = ggplot(model.data, aes(X, Y)) +
  geom_smooth(method='lm', se=FALSE) +
  geom_point(size=model.data$.cooksd*10)
p2 <- ggMarginal(p2, type="boxplot")
p2

## Figure 17.2: Collider bias

In [ ]:
# https://observablehq.com/@herbps10/collider-bias
npts = 1000
set.seed(123456)

conf_df = data.frame(height=rnorm(npts, 175, 20),
                     speed = rnorm(npts, 50, 10)) %>%
  mutate(zheight = scale(height),
         zspeed = scale(speed),
         speedXheight = zheight * zspeed,
         NBA = as.factor(speedXheight > quantile(speedXheight, .90) & speed > mean(speed) & height & mean(height)))

ggplot(conf_df, aes(height, speed, color=NBA)) + geom_point(size=1.5)

In [ ]:
summary(lm(speed ~ height, data=conf_df))

Model with NBA player status (collider)

In [ ]:
summary(lm(speed ~ height + NBA, data=conf_df))

same regression only on the NBA players, where we see a strong negative relationship between speed and height:

In [ ]:
summary(lm(speed ~ height, data=conf_df %>% dplyr::filter(NBA==TRUE)))

## Example 1: Self-regulation and arrest

In [ ]:
behavdata <- read_csv('https://raw.githubusercontent.com/statsthinking21/statsthinking21-figures-data/main/Eisenberg/meaningful_variables.csv',
                      show_col_types = FALSE)
demoghealthdata <- read_csv('https://raw.githubusercontent.com/statsthinking21/statsthinking21-figures-data/main/Eisenberg/demographic_health.csv',
                            show_col_types = FALSE)

# dplyr::recode Sex variable from 0/1 to Male/Female
demoghealthdata <- demoghealthdata %>%
  mutate(Sex = dplyr::recode_factor(Sex, `0`="Male", `1`="Female"))

# combine the data into a single data frame by subcode
alldata <- merge(behavdata, demoghealthdata, by='subcode')

## Table 17.1

In [ ]:
arrestdata <- alldata %>%
  drop_na(ArrestedChargedLifeCount) %>%
  mutate(everArrested = ArrestedChargedLifeCount > 0)

#ggplot(arrestdata,aes(ArrestedChargedLifeCount)) +
#  geom_histogram(bins=max(unique(arrestdata$ArrestedChargedLifeCount)) + 1) # add 1 to account for zeros

arrest_table <- arrestdata %>%
  group_by(ArrestedChargedLifeCount) %>%
  summarize(number = n(),
            proportion = n()/nrow(arrestdata))

kable(arrest_table, caption='Frequency distribution of number of reported arrests in the Eisenberg et al. dataset', digits=3)

## Figure 17.3

In [ ]:
# First, rename the survey variables so that they are shorter, and find all variables related to impulsivity.  We will first rename the variables so taht their names are shorter, for easier display.

rename_list = list('upps_impulsivity_survey' = 'UPPS', 'sensation_seeking_survey' = 'SSS',
                   'dickman_survey' = 'Dickman',  'bis11_survey' = 'BIS11')
impulsivity_variables = c()

for (potential_match in names(alldata)){
  for (n in names(rename_list)){
    if (str_detect(potential_match, n)){
      # print(sprintf('found match: %s %s', n, potential_match))
      replacement_name <- str_replace(potential_match, n, toString(rename_list[n]))
      names(alldata)[names(alldata) == potential_match] <- replacement_name
      impulsivity_variables <- c(impulsivity_variables, replacement_name)
    }
  }
}

impulsivity_data <- alldata[,impulsivity_variables] %>%
  drop_na()
cc = cor(impulsivity_data, use='pairwise.complete')

In [ ]:
colors <- viridis(100)
cellvalues <- format(round(cc, 2), nsmall = 2)
par(mar=c(4,4,4,8)+0.1)
hm <- heatmap.2(cc, trace='none', key=FALSE, dendrogram='row',
                cellnote=cellvalues, notecol='black',
                col=colors, margins=c(10, 14), revC = TRUE)

## Figure 17.4

In [ ]:
# use the clustering from the heatmap to identify the two sets of variables
groups = cutree(hm$rowDendrogram, k=2)
vars_cluster1 <- names(groups[groups == 1])
vars_cluster2 <- names(groups[groups == 2])

# compute the mean impulsivity and sensation seeking values for each individual
# standardize the  measures to  make interpretation of the model easier

alldata <- alldata %>%
  mutate(mean_impulsivity = rowMeans(dplyr::select(., all_of(vars_cluster1)), na.rm = TRUE),
         mean_impulsivity = (mean_impulsivity - mean(mean_impulsivity))/sd(mean_impulsivity),
         mean_senseek = rowMeans(dplyr::select(., all_of(vars_cluster2)), na.rm = TRUE),
         mean_senseek = (mean_senseek - mean(mean_senseek))/sd(mean_senseek))


# create some additional useful variables, and clean up the data frame
# to only include the variables of interest and individuals without missing data
arrestdata <- alldata %>%
  mutate(everArrested = ArrestedChargedLifeCount > 0,
         anyTickets = TrafficTicketsLastYearCount > 0,
         anyAccidents = TrafficAccidentsLifeCount > 0) %>%
  dplyr::select(c(mean_impulsivity, mean_senseek, ArrestedChargedLifeCount,
           everArrested, anyTickets, TrafficTicketsLastYearCount,
           anyAccidents,TrafficAccidentsLifeCount, Age, Sex)) %>%
  drop_na() %>%
  mutate(
    everArrestedInt = as.integer(everArrested),
    everArrested = dplyr::recode_factor(everArrestedInt, `1`='Arrested', `0`='NotArrested'),
    SexInt = dplyr::recode(Sex, Female=0, Male=1))

In [ ]:
pairs.panels(arrestdata[,c('mean_impulsivity', 'mean_senseek')])

## Figure 17.5

In [ ]:
plotdata = pivot_longer(arrestdata,
                        c(mean_impulsivity, mean_senseek),
                        names_to = 'variable') %>%
  mutate(variable = dplyr::recode(variable, mean_impulsivity="Mean impulsivity",
         mean_senseek="Mean sens. seeking"))


p1 = ggplot(plotdata, aes(everArrested, value)) +
  geom_boxplot() +
  geom_beeswarm(alpha=0.1) +
  facet_grid(. ~ variable)

p1 = ggplot(arrestdata, aes(everArrested, mean_impulsivity)) +
  geom_boxplot() +
  geom_beeswarm(alpha=0.1) +
  ylab('Mean impulsivity')
p2 = ggplot(arrestdata, aes(everArrested, mean_senseek)) +
  geom_boxplot() +
  geom_beeswarm(alpha=0.1)+
  ylab('Mean sensation seeking')
plot_grid(p1, p2)

 t-test output for the mean impulsivity scores:

In [ ]:
t.test(mean_impulsivity ~ everArrested, data=arrestdata, alternative='greater')
imp_arrest_d = cohen.d(mean_impulsivity ~ everArrested, data=arrestdata)

t-test output for the sensation seeking variable:

In [ ]:
t.test(mean_senseek ~ everArrested, data=arrestdata, alternative='greater')
imp_arrest_d = cohen.d(mean_senseek ~ everArrested, data=arrestdata)

## Figure 17.6: Logistic regression



In [ ]:
set.seed(1234)
n = 100
b0 = 100
b1 = 3

logreg_df = data.frame(x = rnorm(n, 0,10)) %>%
  mutate(y = as.integer(b0 + b1*x + rnorm(n, 0, 5)),
         y_bin = as.integer(runif(n) < 1/(1 + exp(-(y - mean(y))/sd(y)))))
lm_result_binary = lm(y_bin ~ x, data=logreg_df)
glm_result_binary = glm(y_bin ~ x, family=binomial, data=logreg_df)

p1 = ggplot(logreg_df, aes(x, y_bin)) +
  geom_jitter(height=0.01, width=0) +
  geom_smooth(method='lm', se=FALSE) +
  ggtitle('B) Linear regression')

pred_df = data.frame(x=seq(-25, 25, by=.1))
pred_df = pred_df %>%
  mutate(y = predict(glm_result_binary, newdata=., type='response'))

p2= ggplot(logreg_df, aes(x, y_bin)) +
  geom_jitter(height=0.01, width=0) +
  geom_line(data=pred_df, aes(x,y), color='blue') +
  ggtitle('B) Logistic regression')

plot_grid(p1, p2)

Logistic regression output:

In [ ]:
summary(glm_result_binary)

Logistic regression output for impulsivity dataset

In [ ]:
impmodeldata <- arrestdata %>%
  dplyr::select(everArrestedInt, mean_impulsivity, mean_senseek, Age, Sex) %>%
  mutate(AgeSquared = (Age - mean(Age))**2)

# glm requires int or logical variable as Y
glm_imp_arrest = glm(everArrestedInt ~ mean_impulsivity + mean_senseek + Age +  AgeSquared + Sex,
                    data=impmodeldata, family=binomial)
glm_imp_arrest_baseline = glm(everArrestedInt ~ Age + Sex,
                    data=impmodeldata, family=binomial)
probabilities <- predict(glm_imp_arrest, type = "response")
predicted.classes <- ifelse(probabilities > 0.5, "pos", "neg")
model.data = augment(glm_imp_arrest)

glm_imp_arrest
# summary(glm_imp_arrest)

Test for overdispersion

In [ ]:
testDispersion(glm_imp_arrest, alternative='greater', type='PearsonChisq')

### Figure 17.7

In [ ]:
cat_logit_est = function(iv){
  ncats = min(18, round(length(unique(iv))/2))
  iv_cut = cut_number(iv, ncats)
  mod = glm(everArrestedInt ~ iv_cut, family=binomial, data=impmodeldata)
  return(mod$linear.predictors)
}

loess_logit_est = function(iv, span=1){
  fit = loess(everArrestedInt ~ iv, data=impmodeldata, span=span)$fitted
  fit[fit<=0] = 0 #negatives can happen at edges
  logit_of_fit = logit(fit)
  return(logit_of_fit)
}

# pull out continuous predictors
predictors <- colnames(model.data %>% dplyr::select(-everArrestedInt, -Sex, -starts_with('.')))

use_glm = FALSE
if (use_glm){
  model_string = "Using glm() to estimate logit"
  model.data = model.data %>%
    mutate(observed_logit_Age = cat_logit_est(Age),
           observed_logit_senseek = cat_logit_est(mean_senseek),
           observed_logit_impuls =  cat_logit_est(mean_impulsivity))
} else {
  model_string = "Using loess() to estimate logit"
  model.data = model.data %>%
    mutate(observed_logit_Age = loess_logit_est(Age),
           observed_logit_senseek = loess_logit_est(mean_senseek),
           observed_logit_impuls = loess_logit_est(mean_impulsivity))
}


point_alpha=.3
jitter_width=0
jitter_height=.1
p1  = ggplot(model.data, aes(Age,observed_logit_Age,  size=.cooksd)) +
  geom_jitter(width=jitter_width, height=jitter_height, alpha=point_alpha) +
  geom_smooth(se=FALSE, span=1) +
  ggtitle(model_string)+
  theme(legend.position="none")

p2 = ggplot(model.data, aes(mean_senseek,observed_logit_senseek,  size=.cooksd)) +
  geom_jitter(width=jitter_width, height=jitter_height, alpha=point_alpha) +
  geom_smooth(se=FALSE, span=1)+
  theme(legend.position="none")

p3 = ggplot(model.data, aes(mean_impulsivity,observed_logit_impuls, size=.cooksd)) +
  geom_jitter(width=jitter_width, height=jitter_height, alpha=point_alpha) +
  geom_smooth(se=FALSE,span=1)

plot_grid(p1, p2, p3)
# ggplot(impmodeldata_long_sample, aes(sample_logit, .fitted)) +
#   geom_point() +
#   facet_wrap(~predictors, scales = "free_x") +
#   geom_smooth(se=FALSE)
#
#

model including $Age^2$ alongside Age

In [ ]:
glm_imp_arrest2 = glm(everArrestedInt ~ mean_impulsivity + mean_senseek + Age +  AgeSquared + Sex,
                    data=impmodeldata, family=binomial)
glm_imp_arrest_baseline2 = glm(everArrestedInt ~ Age + AgeSquared + Sex,
                    data=impmodeldata, family=binomial)

summary(glm_imp_arrest2)

### Table 17.2

In [ ]:
oddsratios = exp(cbind("Odds ratio" = coef(glm_imp_arrest), confint.default(glm_imp_arrest, level = 0.95)))[2:6,]
kable(oddsratios, digits=3, caption='Effect sizes for each of the variables in the logistic regression model, expressed as odds ratios, along with 95 percent confidence limits for each odds ratio.')

Bayes factor using approximation via BIC

In [ ]:
BF_01 = exp((BIC(glm_imp_arrest2) - BIC(glm_imp_arrest_baseline2))/2)  # BICs to Bayes factor
# 1/BF_01

## Example 2: Mask-wearing and face-touching

### Figure 17.8

In [ ]:
# data cleaning based on .do file distributed with data

maskdata = read_csv('https://raw.githubusercontent.com/statsthinking21/statsthinking21-figures-data/main/mask_wearing/DataVersion2/MaskFaceTouchOSF.csv') %>%
  filter(face_touching != "Missing") %>%
  mutate(face_touching = dplyr::recode(face_touching, 'Yes'=1, 'No'=0),
         mask_front_touching = dplyr::recode(mask_front_touching, 'Yes'=1, .default=0),
         mask_strap_touching = dplyr::recode(mask_strap_touching, 'Yes'=1,.default=0),
         face_touching = face_touching | mask_front_touching | mask_strap_touching,
         mask_wearing = dplyr::recode(mask_YesNo, 'Yes'=TRUE, 'No'=FALSE),
         age_std = scale(age),
         study = as.factor(study),
         segment_crowding_std = scale(segment_crowding)) %>%
  filter(change==0 & non_covering != 'Yes')

maskdata_study1 = maskdata %>%
  filter(study==1)

maskdata_study2 = maskdata %>%
  filter(study==2)

In [ ]:
ggplot(maskdata %>% mutate(study=dplyr::recode(study, `1`='Study 1', `2`='Study 2'))
       , aes(x=mask_wearing, fill=face_touching)) + geom_bar() + facet_grid(. ~ study)

simple chi-squared test

In [ ]:
chisq.test(maskdata_study1$mask_wearing, maskdata_study1$face_touching)

analogous test for the data from the second study

In [ ]:
chisq.test(maskdata_study2$mask_wearing, maskdata_study2$face_touching)

Logistic regression  model including duration and study along with mask wearing

In [ ]:
glm_result_combined = glm(face_touching ~ mask_wearing + duration_of_observation + study,
                          family=binomial, data=maskdata)
summary(glm_result_combined)

Mixed effects model

In [ ]:
glmer_result = glmer(face_touching ~ mask_wearing + duration_of_observation + (1 + mask_wearing  |unique_situation), data=maskdata, family=binomial)
glmer_result_baseline = update(glmer_result, formula = ~ . -mask_wearing)  # Without mask effect term
summary(glmer_result)

maskdata = maskdata %>%
  mutate(glmer_resid = residuals(glmer_result),
         )

test for overdispersion:

In [ ]:
testDispersion(glmer_result, alternative='greater', type='PearsonChisq')

### Figure 17.9

In [ ]:
#alt.est.b <- influence(glmer_result, obs=TRUE) #"unique_situation")
#cd = cooks.distance(alt.est.b)

loess_logit_est2 = function(iv, span=1){
  fit = loess(face_touching_int ~ iv, data=maskmodeldata, span=span)$fitted
  fit[fit<=0] = 0 #negatives can happen at edges
  logit_of_fit = logit(fit)
  return(logit_of_fit)
}


maskmodeldata = augment(glmer_result) %>%
  mutate(face_touching_int = as.integer(face_touching))
maskmodeldata = maskmodeldata %>%
  mutate(observed_logit = loess_logit_est2(duration_of_observation))

ggplot(maskmodeldata, aes(duration_of_observation, observed_logit, size=.cooksd))+
  geom_jitter() +
  geom_smooth(method = "loess", se=FALSE)

### Table 17.3

In [ ]:
effect_table = tidy(glmer_result,conf.int=TRUE,exponentiate=TRUE,effects="fixed") %>%
  dplyr::select(term, estimate, conf.low, conf.high)

names(effect_table) = c('', 'Odds ratio', '2.5 %', '97.5 %')
kable(effect_table[2:3, ], digits=3, caption = 'Odds ratios and confidence intervals for the independent variables in the mask wearing study. Note that odds ratios are only presented for the fixed effects; since video segments were modeled as a random effect, that variable is not included.')

## Example 3: Asthma and air pollution

#### 3. Prepare the data for analysis

In [ ]:
# asthma data
# https://chronicdata.cdc.gov/500-Cities-Places/500-Cities-Census-Tract-level-Data-GIS-Friendly-Fo/5mtz-k78d (for 2014)
#
# PM2.5 data:
#
# filtered for 2014 and averaged ds_pm_pred using:
#
# https://data.cdc.gov/Environmental-Health-Toxicology/Daily-Census-Tract-Level-PM2-5-Concentrations-2011/fcqm-xrf4/data


pmdata = read_csv('https://raw.githubusercontent.com/statsthinking21/statsthinking21-figures-data/main/CDC_PM2.5_2014/Daily_Census_Tract-Level_PM2.5_mean_2014.csv') %>%
  rename(TractFIPS=ctfips,
         pm25_mean=ds_pm_pred)

asthmadata = read_csv('https://raw.githubusercontent.com/statsthinking21/statsthinking21-figures-data/main/500cities_disease/acsdata_with_censusdata.csv') %>%
  dplyr::select(-ends_with('_Crude95CI'))

pm_asthma_data = inner_join(asthmadata, pmdata, by='TractFIPS') %>%
  drop_na() %>%
  rename(asthma_prev=CASTHMA_CrudePrev,
         city=PlaceFIPS) %>%
  mutate(MedianIncome = MedianIncome/1000,
         Population2010 = Population2010/1000)
  # mutate(log_asthma_prev = log(asthma_prev))

### Figure 17.10

In [ ]:
pairs.panels(pm_asthma_data %>% dplyr::select(pm25_mean, asthma_prev, MedianIncome, MedianAge, Population2010))

standard linear regression model

In [ ]:
lm_asthma_pm = lm(asthma_prev ~ pm25_mean + MedianIncome +  MedianAge + Population2010, data=pm_asthma_data)
summary(lm_asthma_pm)

### Figure 17.11

In [ ]:
model.data = augment(lm_asthma_pm)

ggplot(model.data, aes(sample=.resid)) +
  geom_qq() +
  geom_qq_line()

### Figure 17.12

In [ ]:
model.data = model.data %>%
  mutate(city = pm_asthma_data$city)

model_means_sorted = model.data %>% group_by(city) %>%
  summarize_all(mean) %>%
  arrange(.resid)

p1 = ggplot(model.data, aes(y=.resid, group=factor(city, levels=model_means_sorted$city))) +
  geom_boxplot(outlier.size=0.5, alpha=.2, size=.1)

lmer_asthma_pm = lmer(asthma_prev ~ pm25_mean + MedianAge + MedianIncome + Population2010 + (1 + pm25_mean|city),
                    data=pm_asthma_data)
lmer_asthma_pm_baseline = update(lmer_asthma_pm, formula = ~ . -pm25_mean)

pm_asthma_data = pm_asthma_data %>%
  mutate(lmer_resid = resid(lmer_asthma_pm))

p2 = ggplot(pm_asthma_data, aes(y=lmer_resid, group=city)) +
  geom_boxplot(outlier.size=0.5, alpha=.2, size=.1)

plot_grid(p1, p2, nrow=2)

linear mixed effect model output:

In [ ]:
summary(lmer_asthma_pm)

### Table 17.4

In [ ]:
ci = confint(lmer_asthma_pm)
lmer_confint = cbind(fixef(lmer_asthma_pm), ci[5:9,])[2:5, ]

kable(lmer_confint, digits=2, caption='Parameter estimates and confidence intervals for regression parameters from mixed effect model of asthma prevalence.')

Compute delta r-squared

In [ ]:
r2 = r.squaredGLMM(lmer_asthma_pm)
# r2
r2_baseline = r.squaredGLMM(lmer_asthma_pm_baseline)
# r2_baseline

delta_r2 = r2[1] - r2_baseline[1] [1] # marginal r-squared, reflecting fixed effects only
# delta_r2

lmer_asthma_pm_noincome = update(lmer_asthma_pm, formula = ~ . -MedianIncome)
r2_noincome = r.squaredGLMM(lmer_asthma_pm_noincome)
delta_r2_noincome = r2[1] - r2_noincome[1] [1] # marginal r-squared, reflecting fixed effects only
#delta_r2_noincome

## Example 4: Response of plants to nitrogen fertilizers and soil tilling

#### 3. Prepare and visualize the data

In [ ]:
fert_data_all = read_delim('https://raw.githubusercontent.com/statsthinking21/statsthinking21-figures-data/main/fertilizer/Fertsyntraitsall_Jan27_2008.txt', delim='\t')

fert_data = fert_data_all %>%
  dplyr::filter(Rawabundmetric == "Grams biomass") %>%
  dplyr::filter(Site == 'KBS') %>%
  mutate(YearFac = as.factor(Year),
         plotID_common = plotID - Fert,
         Fert = as.factor(Fert),
         log_rawabund = as.numeric(scale(log(Rawabund))))


### Figure 17.13

In [ ]:
ggplot(fert_data, aes(Rawabund)) +
  geom_histogram(bins=100) +
  xlab('Raw crop abundance')


Simple linear model output:

In [ ]:
lm.result_fert = lm(Rawabund ~ Fert*Experiment + Year, data=fert_data)
summary(lm.result_fert)

### Figure 17.14

In [ ]:
model.data = augment(lm.result_fert) %>%
  mutate(plotID_common = fert_data$plotID_common,
         Species_code = fert_data$Species_code)

p1 = ggplot(model.data, aes(sample=.resid)) + geom_qq() + geom_qq_line()

lmer.result_fert = lmer(log_rawabund ~ Fert*Experiment + Year + (1 + Fert|plotID_common) + (1 + Fert|Species_code) , data=fert_data)

fert_data = fert_data %>%
  mutate(lmer_resid = resid(lmer.result_fert))

p2 = ggplot(fert_data, aes(sample=lmer_resid)) + geom_qq() + geom_qq_line()
plot_grid(p1, p2)

### Figure 17.15

In [ ]:
model_means_sorted = model.data %>% group_by(Species_code) %>%
  summarize_all(mean) %>%
  arrange(.resid)
ggplot(model.data, aes(y=.resid, group=factor(Species_code, levels=model_means_sorted$Species_code))) + geom_boxplot()

Linear mixed effects model output:

In [ ]:
summary(lmer.result_fert)

Effect sizes

In [ ]:
emm = emmeans(lmer.result_fert, pairwise ~ Fert*Experiment)
contrast(emm, 'tukey')

Compute delta r-squared values

In [ ]:
lmer.result_fert_noint = update(lmer.result_fert, formula = ~ . -Fert:Experiment)
lmer.result_fert_null = update(lmer.result_fert, formula = ~ . -Fert*Experiment)

r2 = r.squaredGLMM(lmer.result_fert)
#r2
r2_noint = r.squaredGLMM(lmer.result_fert_noint)
#r2_noint
r2_null = r.squaredGLMM(lmer.result_fert_null)
#r2_null

delta_r2_int = r2[1] - r2_noint[1] [1] # marginal r-squared, reflecting fixed effects only
#delta_r2_int

delta_r2_full = r2[1] - r2_null[1] [1] # marginal r-squared, reflecting fixed effects only
#delta_r2_full